In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import cupy as cp
EPSILON=1e-8

In [ ]:
def plot_random_sample_and_recovery(random_sample,recovered_random_sample):
    # Generate indices for plotting
    indices = np.arange(random_sample.shape[0])
    frequencies = np.fft.fftfreq(random_sample.shape[-1])

    # Create a dataframe for the original and recovered random samples for easy plotting with Plotly
    df_samples = pd.DataFrame({
        'Index': np.concatenate([indices, indices]),
        'Value': np.concatenate([random_sample, recovered_random_sample.real]),
        'Sample Type': ['Original' for _ in range(len(random_sample))] + ['Recovered' for _ in range(len(recovered_random_sample))]
    })

    # Plot original and recovered random sample using Plotly Express
    fig_samples = px.line(df_samples, x='Index', y='Value', color='Sample Type', title='Original vs. Recovered Random Sample')
    return fig_samples

In [ ]:
def plot_frequencies(random_sample,fft_result_random):
    # Create a dataframe for the FFT magnitude plot for easy plotting with Plotly
    frequencies=np.fft.fftfreq(random_sample.shape[-1])
    df_fft = pd.DataFrame({
        'Frequency': frequencies,
        'Magnitude': np.abs(fft_result_random)
    })

    # Plot FFT magnitude of random sample using Plotly Express
    fig_fft = px.line(df_fft, x='Frequency', y='Magnitude', title='FFT Magnitude of Random Sample')
    return fig_fft

In [ ]:
# Generate a random sample
np.random.seed(42)  # For reproducibility
sample_size=1024
random_sample = np.random.randn(sample_size)  # Generate 1024 random numbers from a normal distribution

In [ ]:
# Perform FFT on the random sample
fft_result_random = np.fft.fft(random_sample)
# Perform IFFT to recover the original random sample
recovered_random_sample = np.fft.ifft(fft_result_random)

In [ ]:
plot_random_sample_and_recovery(random_sample,recovered_random_sample)

In [ ]:
plot_frequencies(random_sample,fft_result_random)

In [ ]:
#* Implementations emphasizing readability over performance-- for performance use np.ftt on CPU or cp.ftt on GPU. We check that our functions' outputs agree with the high performance ones.

def my_fft(x):
    """
    Compute the Discrete Fourier Transform (DFT) of the 1D array x.
    """
    N = len(x)
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(-2j * np.pi * k * n / N)
    return np.dot(e, x)

def my_ifft(X):
    """
    Compute the Inverse Discrete Fourier Transform (IDFT) of the 1D array X.
    """
    N = len(X)
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(2j * np.pi * k * n / N)
    return np.dot(e, X) / N

def fftfreq(N, d=1.0):
    """
    Generate the array of frequencies for an FFT output of size N.
    
    Parameters:
    - N (int): The window length, or number of points in the FFT.
    - d (float): The sample spacing (inverse of the sampling rate).
    
    Returns:
    - f (ndarray): An array of frequency bins.
    """
    val = 1.0 / (N * d)
    results = np.empty(N, dtype=int)
    N_half = N // 2
    results[:N_half] = np.arange(0, N_half)
    results[N_half:] = np.arange(-N_half, 0, dtype=int)
    return results * val

In [ ]:
# verify this fftfreq function matches numpy/cupy 
assert np.abs(fftfreq(sample_size)-np.fft.fftfreq(random_sample.shape[-1])).max()<EPSILON

In [ ]:
my_fft_result_random = my_fft(random_sample)

# Perform IFFT to recover the original random sample
my_recovered_random_sample = np.array(my_ifft(fft_result_random))

In [ ]:
plot_random_sample_and_recovery(random_sample,my_recovered_random_sample)

In [ ]:
plot_frequencies(random_sample,my_fft_result_random)

In [ ]:
dft_result_random = my_fft(random_sample)

# Perform IFFT to recover the original random sample
dft_recovered_random_sample = np.array(my_ifft(fft_result_random))

In [ ]:
plot_random_sample_and_recovery(random_sample,dft_recovered_random_sample)

# Part Two

In [ ]:
inner_sims=2048
outer_sims=100000

In [ ]:
%%time
cp.random.seed(42)
rands=cp.random.randn(inner_sims,outer_sims)

In [ ]:
%%time
column_sums=rands.sum(axis=0)
column_sums.shape

In [ ]:
%%time
fft=cp.fft.fft(rands,axis=0)
fft.shape

In [ ]:
%%time
# first frequency always corresponds with sum of all values
assert np.abs(column_sums-fft[0]).max()<EPSILON

In [ ]:
mean_frequency_domain=fft.mean(axis=1)
mean_frequency_domain.shape

In [ ]:
%%time
# the mean of lowest frequency is the mean of column sums
assert np.abs(mean_frequency_domain[0]-column_sums.mean())<EPSILON

In [ ]:
column_sums.mean()/inner_sims

In [ ]:
%%time
ifft_mean_fft=cp.fft.ifft(mean_frequency_domain)

In [ ]:
%%time
# the ifft of the mean of the fft is the mean of the original random matrix
assert np.abs(ifft_mean_fft-rands.mean(axis=1)).max()<EPSILON

In [ ]:
fig=px.scatter(ifft_mean_fft.real.get(),opacity=0.3)
fig.update_traces(marker=dict(size=10))
fig

In [ ]:
px.histogram(ifft_mean_fft.real.get(),nbins=50)

In [ ]:
fig=px.violin(ifft_mean_fft.real.get(),points='all',box=True)
fig.update_traces(marker=dict(size=10,opacity=0.1))
fig